In [1]:
import keyring
import pandas as pd
import datetime
import time
import pymssql
from selenium import webdriver
from urllib.request import urlopen
from pymongo import MongoClient

In [2]:
# Get dates for query
today = datetime.datetime.today()
one_week_ago = today - datetime.timedelta(days=7)

In [3]:
# Create connection string
user = "dbadmin"
password = keyring.get_password('docdb_prod_dbadmin', 'dbadmin')
host = "production-documentdb.cluster-cb6kajicuplh.us-east-1.docdb.amazonaws.com"
port = "27017"
options = "tls=true&tlsAllowInvalidCertificates=true&replicaSet=rs0&readPreference=secondaryPreferred&retryWrites=false"
cxn_string_template= "mongodb://{}:{}@{}:{}/?{}"
cxn_string_prod = cxn_string_template.format(user, password, host, port, options)


# Get past week's new products

In [4]:
# Get past week's new products
client = MongoClient(cxn_string_prod)
db = client['product-prod']
results_ = list(db.PdwProductCore.find({'createTimestamp': {'$gte': one_week_ago}}, 
                                       ('productGeneral.cusip', 'productGeneral.isin')))
cusips = [i['productGeneral']['cusip'] for i in results_ if 'cusip' in i['productGeneral']]
isins = [i['productGeneral']['isin'] for i in results_ if 'isin' in i['productGeneral']]

# Get BMO products added within first 5 pages

In [96]:
# # Setup for bmo
# bmo_act_dict = {}
# chrome_path = r"/opt/homebrew/bin/chromedriver"
# op = webdriver.ChromeOptions()
# op.add_argument('--headless')
# urls = [
#     'https://www.bmonotes.com/Type/PPNs#active', 
#     'https://www.bmonotes.com/Type/Fixed-Income-Notes#active', 
#     'https://www.bmonotes.com/Type/NPPNs#active'
# ]
# all_bmo_active_products = pd.DataFrame()
# for url in urls:
#     print(url)
#     # Get first page
#     driver = webdriver.Chrome(chrome_path, options=op)
#     page = driver.get(url)
#     time.sleep(2)
#     bmo_act_dict[0] = pd.read_html(driver.page_source)[1]
#     # Get remaining pages
#     flag = True
#     num = 1
#     while flag:
#         if 'disabled' in driver.find_element_by_id("DataTables_Table_1_next").get_attribute('class'):
#             flag = 0
#         else:
#             driver.find_element_by_id("DataTables_Table_1_next").click()
#             time.sleep(2)
#             bmo_act_dict[num] = pd.read_html(driver.page_source)[1]
#             num += 1
#     driver.close()
#     # Combine the dataframes
#     bmo_active_products = pd.concat([bmo_act_dict[k] for k in bmo_act_dict.keys()], ignore_index=True)
#     all_bmo_active_products = pd.concat([all_bmo_active_products, bmo_active_products], ignore_index=True)
# # Remove junk
# all_bmo_active_products = all_bmo_active_products[(all_bmo_active_products['JHN Code / Cusip'].isna()==False) & (all_bmo_active_products['JHN Code / Cusip']!='Loading...')]
# # Create pdw cusip for comparison
# listy = []
# for i in all_bmo_active_products['JHN Code / Cusip']:
#     if 'JHN' in str(i):
#         if len(str(i)) == 7:
#             listy.append('CA' + str(i))
#         elif len(str(i)) == 8:
#             listy.append('C' + str(i))
#         elif len(str(i)) == 6:
#             listy.append('CAA' + str(i))
#         else:
#             listy.append('Error')
#     else:
#         if len(str(i)) == 9:
#             listy.append(str(i))
#         else:
#             listy.append('Error')
# all_bmo_active_products['pdwCusip'] = listy
# # Filter out any products already present in pdw
# new_bmo_active_products = all_bmo_active_products[~all_bmo_active_products['JHN Code / Cusip'].isin(cusips) | 
#                                               all_bmo_active_products['JHN Code / Cusip'].isin(isins)]

In [ ]:
pd.Series([len(str(i)) for i in all_bmo_active_products['JHN Code / Cusip']]).value_counts()

7     2769
8     1885
6      272
9       14
3        1
10       1
dtype: int64

In [84]:
all_bmo_active_products.drop_duplicates()

,Note Name,JHN Code / Cusip,Issue Date,Currency,Term,Maturity Date,Current Bid Price,Product Type,pdwCusip
0,Bank of Montreal S&P 500 Daily Risk Control In...,JHN2848,"Aug 15, 2022",CAD,7.0,"Aug 15, 2029",$100.00,NaN,CAJHN2848
1,Bank of Montreal S&P 500 Daily Risk Control In...,JHN2849,"Aug 15, 2022",CAD,5.0,"Aug 16, 2027",$100.00,NaN,CAJHN2849
2,Bank of Montreal Core Investor Volatility (5%)...,JHN4165,"Aug 12, 2022",CAD,5.0,"Aug 12, 2027",$99.85,NaN,CAJHN4165
3,Bank of Montreal Core Investor Volatility (5%)...,JHN4166,"Aug 12, 2022",CAD,3.0,"Aug 12, 2025",$99.87,NaN,CAJHN4166
4,Bank of Montreal S&P/TSX 60 Canadian Growth Pr...,JHN2846,"Aug 11, 2022",CAD,4.5,"Feb 11, 2027",$100.41,NaN,CAJHN2846
...,...,...,...,...,...,...,...,...,...
4937,Bank of Montreal US Q-Model Principal At Risk ...,JHN8068,"Jun 19, 2013",CAD,10.0,"Jun 19, 2023",$157.23,NaN,CAJHN8068
4938,Bank of Montreal Canadian Q-Model Principal At...,JHN8066,"Jun 19, 2013",CAD,10.0,"Jun 19, 2023",$126.85,NaN,CAJHN8066
4939,Bank of Montreal Canadian Q-Model Principal At...,JHN8067,"Jun 19, 2013",CAD,10.0,"Jun 19, 2023",$160.88,NaN,CAJHN8067
4940,Bank of Montreal Canadian Q-Model Principal At...,JHN8055,"Oct 3, 2012",CAD,10.0,"Oct 3, 2022",$153.96,NaN,CAJHN8055


# Testing NBCSS

In [5]:
# # Setup for nbc_ss
# nbc_ss_act_dict = {}
# chrome_path = r"/opt/homebrew/bin/chromedriver"
# op = webdriver.ChromeOptions()
# op.add_argument('--headless')
# driver = webdriver.Chrome(chrome_path, options=op)
# url = 'https://www.nbcstructuredsolutions.ca/listeProduits.aspx?mode=previous'
# # Get first page
# page = driver.get(url)
# time.sleep(2)
# nbc_ss_act_dict[0] = pd.read_html(driver.page_source)[0]
# nbc_ss_act_dict[0]['urls'] = [driver.find_element_by_xpath('//*[@id="ctl00_cphMain_lvProducts_ctrl{}_lnkProduit"]'.format(i)).get_attribute('href') for i in range(len(nbc_ss_act_dict[0]))]
# # Get remaining 4 pages
# for num in range(2, 6):
#     driver.find_element_by_xpath('//*[@id="ctl00_cphMain_dpProductsHaut"]/a[{}]'.format(str(num))).click()
#     time.sleep(2)
#     nbc_ss_act_dict[num] = pd.read_html(driver.page_source)[0]
#     nbc_ss_act_dict[num]['urls'] = [driver.find_element_by_xpath('//*[@id="ctl00_cphMain_lvProducts_ctrl{}_lnkProduit"]'.format(i)).get_attribute('href') for i in range(len(nbc_ss_act_dict[num]))]
# driver.close()
# # Combine the dataframes
# nbc_ss_active_products = pd.concat([nbc_ss_act_dict[k] for k in nbc_ss_act_dict.keys()], ignore_index=True)


In [6]:
# nbc_ss_active_products

# Testing rbcnotes

In [7]:
# rbc_act_dict = {}
# chrome_path = r"/opt/homebrew/bin/chromedriver"
# op = webdriver.ChromeOptions()
# op.add_argument('--headless')
# driver = webdriver.Chrome(chrome_path, options=op)
# url = 'https://www.rbcnotes.com/Products'
# # Get first page
# page = driver.get(url)
# time.sleep(2)
# rbc_act_dict[0] = pd.read_html(driver.page_source)

actual_rbc_data = rbc_act_dict[0][1][(rbc_act_dict[0][1][7].isna()==False) & (rbc_act_dict[0][1][7].str.contains('Day')==False)]
actual_rbc_data.columns = rbc_act_dict[0][0].columns
actual_rbc_data[['Product Name', 'FundSERV Code', 'ADP Code', 'CUSIP', 'End of Day Price', 'Current ETC', 'ETC End Date', 'Issue Date', 'Maturity Date', 'Currency']]

In [60]:
# # Setup for rbc
# rbc_act_dict = {}
# chrome_path = r"/opt/homebrew/bin/chromedriver"
# op = webdriver.ChromeOptions()
# op.add_argument('--headless')
# driver = webdriver.Chrome(chrome_path, options=op)
# url = 'https://www.rbcnotes.com/Products'
# # Get first page
# page = driver.get(url)
# time.sleep(2)
# temp_table = pd.read_html(driver.page_source)
# temp_data = temp_table[1][(temp_table[1][7].isna()==False) & (temp_table[1][7].str.contains('Day')==False)]
# temp_data.columns = temp_table[0].columns
# rbc_act_dict[0] = temp_data
# rbc_act_dict[0]['urls'] = [driver.find_element_by_xpath('//*[@id="productGrid"]/div[2]/div[3]/table/tbody/tr[{}]/td[3]/a'.format(i)).get_attribute('href') 
#                                 for i in range(1, len(rbc_act_dict[0])*2, 2)]
# # Get remaining 4 pages
# for num in range(2, 6):
#     # Scroll to bottom of page to avoid cookie consent form
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     driver.find_element_by_xpath('//*[@id="productGrid"]/div[2]/div[4]/a[3]').click()
#     time.sleep(2)
#     temp_table = pd.read_html(driver.page_source)
#     temp_data = temp_table[1][(temp_table[1][7].isna()==False) & (temp_table[1][7].str.contains('Day')==False)]
#     temp_data.columns = temp_table[0].columns
#     rbc_act_dict[num] = temp_data
#     rbc_act_dict[num]['urls'] = [driver.find_element_by_xpath('//*[@id="productGrid"]/div[2]/div[3]/table/tbody/tr[{}]/td[3]/a'.format(i)).get_attribute('href') 
#                                 for i in range(1, len(rbc_act_dict[0])*2, 2)]
# driver.close()
# # Combine the dataframes
# rbc_active_products = pd.concat([rbc_act_dict[k] for k in rbc_act_dict.keys()], ignore_index=True)
# rbc_active_products = rbc_active_products[['Product Name', 'FundSERV Code', 'ADP Code',
#        'CUSIP', 'End of Day Price', 'Current ETC', 'ETC End Date',
#        'Issue Date', 'Maturity Date', 'Currency', 'urls']]


/var/folders/bx/xn15n171733878wqb5wnhvgh0000gp/T/ipykernel_35924/2178530415.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rbc_act_dict[0]['urls'] = [driver.find_element_by_xpath('//*[@id="productGrid"]/div[2]/div[3]/table/tbody/tr[{}]/td[3]/a'.format(i)).get_attribute('href')
/var/folders/bx/xn15n171733878wqb5wnhvgh0000gp/T/ipykernel_35924/2178530415.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rbc_act_dict[num]['urls'] = [driver.find_element_by_xpath('//*[@id="productGrid"]/div[2]/div[3]/tab

In [63]:
rbc_active_products

,Product Name,FundSERV Code,ADP Code,CUSIP,End of Day Price,Current ETC,ETC End Date,Issue Date,Maturity Date,Currency,urls
0,RBC Geared Buffer Solactive Canada Bank 40 AR ...,RBC6573,R044936,NaN,$ 100.00,3.00 %,21-Dec-22,22-Aug-22,09-Aug-29,CAD,https://www.rbcnotes.com/Note/Get?p=d43ff2a0ca...
1,RBC Technology Basket Callable Yield 13.50% Se...,RBC7804,R044969,NaN,NaN,0.00 %,-,19-Aug-22,20-Aug-25,USD,https://www.rbcnotes.com/Note/Get?p=f19a6fc1d8...
2,RBC Technology Basket Callable Yield 11.25% Se...,RBC7803,R044967,NaN,NaN,3.00 %,16-Feb-23,19-Aug-22,20-Aug-25,USD,https://www.rbcnotes.com/Note/Get?p=d483368ab2...
3,RBC U.S. Financial Equity Basket Callable Yiel...,RBC7802,R044948,NaN,NaN,0.00 %,-,19-Aug-22,20-Aug-25,USD,https://www.rbcnotes.com/Note/Get?p=19af8fc2d7...
4,RBC Utilities Basket Callable Yield 8.50% Secu...,RBC7799,R044885,NaN,NaN,0.00 %,-,19-Aug-22,20-Aug-29,CAD,https://www.rbcnotes.com/Note/Get?p=d9f4826630...
...,...,...,...,...,...,...,...,...,...,...,...
95,RBC Solactive Canada Bank 40 AR Index Geared B...,RBC6565,R044231,NaN,$ 99.05,2.75 %,19-Mar-23,21-Jun-22,20-Jun-29,CAD,https://www.rbcnotes.com/Note/Get?p=465c2203a1...
96,RBC Technology Basket Callable Yield 12.00% Se...,OTC,R044350,78013U575,$ 101.90,0.00 %,-,21-Jun-22,21-Jun-27,CAD,https://www.rbcnotes.com/Note/Get?p=f32fa17945...
97,"RBC Principal Protected S&P 500 LEOS (CAD), Se...",OTC,R044351,78013U583,$ 103.92,0.00 %,-,21-Jun-22,10-Jun-27,CAD,https://www.rbcnotes.com/Note/Get?p=043ee4bbbd...
98,RBC Canadian Energy Geared Buffered Autocallab...,RBC8282,R044156,NaN,$ 86.19,0.00 %,-,17-Jun-22,17-Jun-27,CAD,https://www.rbcnotes.com/Note/Get?p=c50486943f...
